In [1]:
import numpy as np, pandas as pd, cvxpy as cp, cvxopt
from collections import Counter
from sklearn.metrics import confusion_matrix

## 1' import data 

In [2]:
train = pd.concat([pd.read_csv("X_labeled.csv"), pd.read_csv("y_labeled.csv")],1,names =None)
train = train.drop(train.columns[0],1)
print(Counter(train["target"]))
#train[(train["target"]==1) & (train["school"]=="GP")].iloc[:,1:-1]

Counter({1: 118, -1: 78})


## 2' Define SVM+ Function

In [3]:
def SVMplus(train,C_pos,C_neg,gamma):
    
    F = train.shape[1]
    
    Xt1_pos = np.matrix(train[(train["target"]==1) & (train["school"]=="GP")].iloc[:,1:-1])
    Yt1_pos = np.array(train[(train["target"]==1) & (train["school"]=="GP")].iloc[:,-1])
    
    Xt1_neg = np.matrix(train[(train["target"]==-1) & (train["school"]=="GP")].iloc[:,1:-1])
    Yt1_neg = np.array(train[(train["target"]==-1) & (train["school"]=="GP")].iloc[:,-1])
    
    Xt2_pos = np.matrix(train[(train["target"]==1) & (train["school"]=="MS")].iloc[:,1:-1])
    Yt2_pos = np.array(train[(train["target"]==1) & (train["school"]=="MS")].iloc[:,-1])
    
    Xt2_neg = np.matrix(train[(train["target"]==-1) & (train["school"]=="MS")].iloc[:,1:-1])
    Yt2_neg = np.array(train[(train["target"]==-1) & (train["school"]=="MS")].iloc[:,-1])
    
    K1_pos = Xt1_pos.shape[0]
    K1_neg = Xt1_neg.shape[0]
    
    K2_pos = Xt2_pos.shape[0]
    K2_neg = Xt2_neg.shape[0]
    
    C1_pos = C_pos*np.ones(K1_pos)
    C1_neg = C_neg*np.ones(K1_neg)
    C2_pos = C_pos*np.ones(K2_pos)
    C2_neg = C_neg*np.ones(K2_neg)
    
    w = cp.Variable(F-2)
    b = cp.Variable()
    w1 = cp.Variable(F-2)
    w2 = cp.Variable(F-2)
    d1 = cp.Variable()
    d2 = cp.Variable()
    
    objective = cp.Minimize(0.5*cp.square(cp.norm(w))+
                            0.5*gamma*(cp.square(cp.norm(w1))+cp.square(cp.norm(w2)))+
                            cp.sum(cp.multiply(C1_pos,Xt1_pos*w1+d1))+
                            cp.sum(cp.multiply(C1_neg,Xt1_neg*w1+d1))+
                            cp.sum(cp.multiply(C2_pos,Xt2_pos*w2+d2))+
                            cp.sum(cp.multiply(C2_neg,Xt2_neg*w2+d2))
                           )
    
    constraints = [cp.multiply(Yt1_pos.T,((Xt1_pos*w+b)+(Xt1_pos*w1+d1)))+(Xt1_pos*w1+d1)-1>=0,
                  (Xt1_pos*w1+d1) >= 0,
                   cp.multiply(Yt1_neg.T,((Xt1_neg*w+b)+(Xt1_neg*w1+d1)))+(Xt1_neg*w1+d1)-1>=0,
                  (Xt1_neg*w1+d1) >= 0,
                  cp.multiply(Yt2_pos.T,((Xt2_pos*w+b)+(Xt2_pos*w2+d2)))+(Xt2_pos*w2+d2)-1>=0,
                  (Xt2_pos*w2+d2) >= 0,
                  cp.multiply(Yt2_neg.T,((Xt2_neg*w+b)+(Xt2_neg*w2+d2)))+(Xt2_neg*w2+d2)-1>=0,
                  (Xt2_neg*w2+d2) >= 0]
    
    prob = cp.Problem(objective,constraints)
    
    try:
        results = prob.solve()
    except:
        results = prob.solve(solver = "ECOS_BB")
        
    return w.value,b.value,w1.value,w2.value,d1.value,d2.value

## 3' Define Training Acc Function

In [4]:
def train_accuracy(w,b,w1,d1,w2,d2,train):
    countin = 0
    for i in range(0,train.shape[0]):
        if train.iloc[i][0] == "GP":
            if (sum(train.iloc[i][1:-1]*w)+b+sum(train.iloc[i][1:-1]*w1)+d1)*train.iloc[i][-1] >0:
                countin += 1
        elif train.iloc[i][0] == "MS":
            if (sum(train.iloc[i][1:-1]*w)+b+sum(train.iloc[i][1:-1]*w2)+d2)*train.iloc[i][-1] >0:
                countin += 1
    return countin/train.shape[0]

## 4' Selecting Best Parameters(C_pos,C_neg,gamma)

In [ ]:
def cmgc(C_1,C_2,g):
    
    train_result = []
    w,b,w1,w2,d1,d2 = SVMplus(train,C_1,C_2,g)
    
    for i in range(0,train.shape[0]):
        if train.iloc[i][0] == "GP" :
            if (sum(train.iloc[i][1:-1]*w)+b+sum(train.iloc[i][1:-1]*w1)+d1) > 0 :
                train_result.append(1)
            else:
                train_result.append(-1)
        else:
            if (sum(train.iloc[i][1:-1]*w)+b+sum(train.iloc[i][1:-1]*w2)+d2) > 0 :
                train_result.append(1)
            else:
                train_result.append(-1)

    return confusion_matrix(train["target"],train_result)


gamma_list = (0.1,0.2,0.5,0.8,1,2,5,10,20,30,40,50,100)

diff_thre = 0.06
lower_thre = 1.72
#upper_thre = 
info = []
candidates = []
for gamma in gamma_list:
    for C_p in 2**np.linspace(-15,15,31):
        for C_n in 2**np.linspace(-15,15,31):
            temp = cmgc(C_p,C_n,gamma)
            acc_sum = temp[0,0]/sum(temp[0,:]) + temp[1,1]/sum(temp[1,:])
            acc_diff = abs(temp[0,0]/sum(temp[0,:]) - temp[1,1]/sum(temp[1,:]))
            info.append([C_p,C_n,gamma,acc_sum,acc_diff])
            if acc_sum >lower_thre and acc_diff<diff_thre:
                print("pos:",C_p,"neg:",C_n,"gamma:",gamma,"acc_sum:",acc_sum,"acc_diff:",acc_diff)
                candidates.append([C_p,C_n,gamma,acc_sum,acc_diff])
pd.DataFrame(info).to_csv("info_SVMplus.csv")

pos: 3.0517578125e-05 neg: 3.0517578125e-05 gamma: 0.1 acc_sum: 1.838548457192525 acc_diff: 0.058887440243372446
pos: 3.0517578125e-05 neg: 6.103515625e-05 gamma: 0.1 acc_sum: 1.8809213385484571 acc_diff: 0.016514558887440245
pos: 3.0517578125e-05 neg: 0.0001220703125 gamma: 0.1 acc_sum: 1.9191655801825294 acc_diff: 0.003911342894393766
pos: 3.0517578125e-05 neg: 0.000244140625 gamma: 0.1 acc_sum: 1.9148196436332028 acc_diff: 0.01738374619730554
pos: 3.0517578125e-05 neg: 0.00048828125 gamma: 0.1 acc_sum: 1.9150369404606693 acc_diff: 0.03368100825727938
pos: 3.0517578125e-05 neg: 0.0009765625 gamma: 0.1 acc_sum: 1.8383311603650587 acc_diff: 0.007822685788787531
pos: 3.0517578125e-05 neg: 0.001953125 gamma: 0.1 acc_sum: 1.8552803129074316 acc_diff: 0.009126466753585305
pos: 3.0517578125e-05 neg: 0.00390625 gamma: 0.1 acc_sum: 1.8765754019991308 acc_diff: 0.004780530204259059
pos: 3.0517578125e-05 neg: 0.0078125 gamma: 0.1 acc_sum: 1.8681008257279443 acc_diff: 0.00369404606692747
pos: 3.

pos: 0.00048828125 neg: 0.015625 gamma: 0.1 acc_sum: 1.9022164276401565 acc_diff: 0.020860495436766602
pos: 0.00048828125 neg: 0.03125 gamma: 0.1 acc_sum: 1.8896132116471098 acc_diff: 0.05910473707083874
pos: 0.00048828125 neg: 0.0625 gamma: 0.1 acc_sum: 1.8639721860060843 acc_diff: 0.03346371142981308
pos: 0.00048828125 neg: 1024.0 gamma: 0.1 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 0.00048828125 neg: 2048.0 gamma: 0.1 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 0.00048828125 neg: 4096.0 gamma: 0.1 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 0.00048828125 neg: 8192.0 gamma: 0.1 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 0.00048828125 neg: 16384.0 gamma: 0.1 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 0.00048828125 neg: 32768.0 gamma: 0.1 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 0.0009765625 neg: 3.0517578125e-05 gamma: 0.1 acc_sum: 1.7916123424598 acc_diff: 0.028900

pos: 0.03125 neg: 3.0517578125e-05 gamma: 0.1 acc_sum: 1.7742285962624944 acc_diff: 0.05627987831377668
pos: 0.03125 neg: 6.103515625e-05 gamma: 0.1 acc_sum: 1.7742285962624944 acc_diff: 0.05627987831377668
pos: 0.03125 neg: 0.0001220703125 gamma: 0.1 acc_sum: 1.7742285962624944 acc_diff: 0.05627987831377668
pos: 0.03125 neg: 0.000244140625 gamma: 0.1 acc_sum: 1.7998696219035202 acc_diff: 0.030638852672751016
pos: 0.03125 neg: 0.00048828125 gamma: 0.1 acc_sum: 1.8298565840938723 acc_diff: 0.01629726205997395
pos: 0.03125 neg: 0.0078125 gamma: 0.1 acc_sum: 1.8896132116471098 acc_diff: 0.05910473707083874
pos: 0.03125 neg: 0.0625 gamma: 0.1 acc_sum: 1.876792698826597 acc_diff: 0.04628422425032597
pos: 0.03125 neg: 0.125 gamma: 0.1 acc_sum: 1.8683181225554106 acc_diff: 0.054758800521512385
pos: 0.03125 neg: 1024.0 gamma: 0.1 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 0.03125 neg: 2048.0 gamma: 0.1 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 0.03125 n

pos: 4.0 neg: 6.103515625e-05 gamma: 0.1 acc_sum: 1.8850499782703172 acc_diff: 0.013255106475445477
pos: 4.0 neg: 0.0001220703125 gamma: 0.1 acc_sum: 1.8850499782703172 acc_diff: 0.013255106475445477
pos: 4.0 neg: 0.000244140625 gamma: 0.1 acc_sum: 1.8850499782703172 acc_diff: 0.013255106475445477
pos: 4.0 neg: 0.00048828125 gamma: 0.1 acc_sum: 1.89787049109083 acc_diff: 0.0004345936549327023
pos: 4.0 neg: 0.0009765625 gamma: 0.1 acc_sum: 1.8850499782703172 acc_diff: 0.013255106475445477
pos: 4.0 neg: 0.001953125 gamma: 0.1 acc_sum: 1.89787049109083 acc_diff: 0.0004345936549327023
pos: 4.0 neg: 0.00390625 gamma: 0.1 acc_sum: 1.89787049109083 acc_diff: 0.0004345936549327023
pos: 4.0 neg: 0.0078125 gamma: 0.1 acc_sum: 1.89787049109083 acc_diff: 0.0004345936549327023
pos: 4.0 neg: 0.015625 gamma: 0.1 acc_sum: 1.89787049109083 acc_diff: 0.0004345936549327023
pos: 4.0 neg: 0.03125 gamma: 0.1 acc_sum: 1.9022164276401565 acc_diff: 0.020860495436766602
pos: 4.0 neg: 0.0625 gamma: 0.1 acc_sum: 

pos: 32.0 neg: 0.0009765625 gamma: 0.1 acc_sum: 1.8763581051716645 acc_diff: 0.055845284658843974
pos: 32.0 neg: 0.001953125 gamma: 0.1 acc_sum: 1.8763581051716645 acc_diff: 0.055845284658843974
pos: 32.0 neg: 0.00390625 gamma: 0.1 acc_sum: 1.8763581051716645 acc_diff: 0.055845284658843974
pos: 32.0 neg: 0.0078125 gamma: 0.1 acc_sum: 1.8763581051716645 acc_diff: 0.055845284658843974
pos: 32.0 neg: 0.015625 gamma: 0.1 acc_sum: 1.8763581051716645 acc_diff: 0.055845284658843974
pos: 32.0 neg: 0.03125 gamma: 0.1 acc_sum: 1.8763581051716645 acc_diff: 0.055845284658843974
pos: 32.0 neg: 0.0625 gamma: 0.1 acc_sum: 1.8763581051716645 acc_diff: 0.055845284658843974
pos: 32.0 neg: 0.125 gamma: 0.1 acc_sum: 1.8891786179921772 acc_diff: 0.04302477183833109
pos: 32.0 neg: 0.25 gamma: 0.1 acc_sum: 1.8807040417209908 acc_diff: 0.03455019556714467
pos: 32.0 neg: 0.5 gamma: 0.1 acc_sum: 1.8807040417209908 acc_diff: 0.03455019556714467
pos: 32.0 neg: 1.0 gamma: 0.1 acc_sum: 1.8807040417209908 acc_diff: 

...

pos: 0.015625 neg: 2048.0 gamma: 0.5 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 0.015625 neg: 4096.0 gamma: 0.5 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 0.015625 neg: 8192.0 gamma: 0.5 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 0.015625 neg: 16384.0 gamma: 0.5 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 0.015625 neg: 32768.0 gamma: 0.5 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 0.03125 neg: 3.0517578125e-05 gamma: 0.5 acc_sum: 1.8342025206431987 acc_diff: 0.03759235115167314
pos: 0.03125 neg: 6.103515625e-05 gamma: 0.5 acc_sum: 1.8342025206431987 acc_diff: 0.03759235115167314
pos: 0.03125 neg: 0.0001220703125 gamma: 0.5 acc_sum: 1.8342025206431987 acc_diff: 0.03759235115167314
pos: 0.03125 neg: 0.000244140625 gamma: 0.5 acc_sum: 1.8257279443720122 acc_diff: 0.04606692742285967
pos: 0.03125 neg: 0.00048828125 gamma: 0.5 acc_sum: 1.838548457192525 acc_diff: 0.058887440243372446
pos: 0.031

pos: 1.0 neg: 4.0 gamma: 0.5 acc_sum: 1.9193828770099957 acc_diff: 0.05497609734897879
pos: 1.0 neg: 1024.0 gamma: 0.5 acc_sum: 1.957627118644068 acc_diff: 0.0423728813559322
pos: 1.0 neg: 2048.0 gamma: 0.5 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 1.0 neg: 4096.0 gamma: 0.5 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 1.0 neg: 8192.0 gamma: 0.5 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 1.0 neg: 16384.0 gamma: 0.5 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 1.0 neg: 32768.0 gamma: 0.5 acc_sum: 1.9491525423728815 acc_diff: 0.05084745762711862
pos: 2.0 neg: 0.00048828125 gamma: 0.5 acc_sum: 1.8252933507170797 acc_diff: 0.05606258148631038
pos: 2.0 neg: 0.0009765625 gamma: 0.5 acc_sum: 1.8381138635375924 acc_diff: 0.043242068665797495
pos: 2.0 neg: 0.001953125 gamma: 0.5 acc_sum: 1.8381138635375924 acc_diff: 0.043242068665797495
pos: 2.0 neg: 0.00390625 gamma: 0.5 acc_sum: 1.8509343763581052 acc_diff: 0.0304215

pos: 16.0 neg: 0.00390625 gamma: 0.5 acc_sum: 1.90199913081269 acc_diff: 0.030204259017818313
pos: 16.0 neg: 0.0078125 gamma: 0.5 acc_sum: 1.90199913081269 acc_diff: 0.030204259017818313
pos: 16.0 neg: 0.015625 gamma: 0.5 acc_sum: 1.90199913081269 acc_diff: 0.030204259017818313
pos: 16.0 neg: 0.03125 gamma: 0.5 acc_sum: 1.8935245545415036 acc_diff: 0.021729682746631895
pos: 16.0 neg: 0.0625 gamma: 0.5 acc_sum: 1.8935245545415036 acc_diff: 0.021729682746631895
pos: 16.0 neg: 0.125 gamma: 0.5 acc_sum: 1.8807040417209908 acc_diff: 0.03455019556714467
pos: 16.0 neg: 0.25 gamma: 0.5 acc_sum: 1.8807040417209908 acc_diff: 0.03455019556714467
pos: 16.0 neg: 0.5 gamma: 0.5 acc_sum: 1.8807040417209908 acc_diff: 0.03455019556714467
pos: 16.0 neg: 1.0 gamma: 0.5 acc_sum: 1.9063450673620164 acc_diff: 0.00890916992611912
pos: 16.0 neg: 2.0 gamma: 0.5 acc_sum: 1.9235115167318557 acc_diff: 0.02520643198609296
pos: 16.0 neg: 4.0 gamma: 0.5 acc_sum: 1.9532811820947416 acc_diff: 0.02107779226423301
pos: 

pos: 128.0 neg: 0.0009765625 gamma: 0.5 acc_sum: 1.9445893089960888 acc_diff: 0.0215123859191656
pos: 128.0 neg: 0.001953125 gamma: 0.5 acc_sum: 1.9445893089960888 acc_diff: 0.0215123859191656
pos: 128.0 neg: 0.00390625 gamma: 0.5 acc_sum: 1.9445893089960888 acc_diff: 0.0215123859191656
pos: 128.0 neg: 0.0078125 gamma: 0.5 acc_sum: 1.9445893089960888 acc_diff: 0.0215123859191656
pos: 128.0 neg: 0.015625 gamma: 0.5 acc_sum: 1.9445893089960888 acc_diff: 0.0215123859191656
pos: 128.0 neg: 0.03125 gamma: 0.5 acc_sum: 1.9445893089960888 acc_diff: 0.0215123859191656
pos: 128.0 neg: 0.0625 gamma: 0.5 acc_sum: 1.9445893089960888 acc_diff: 0.0215123859191656
pos: 128.0 neg: 0.125 gamma: 0.5 acc_sum: 1.9445893089960888 acc_diff: 0.0215123859191656
pos: 128.0 neg: 0.25 gamma: 0.5 acc_sum: 1.9530638852672753 acc_diff: 0.029986962190352018
pos: 128.0 neg: 0.5 gamma: 0.5 acc_sum: 1.9530638852672753 acc_diff: 0.029986962190352018
pos: 128.0 neg: 1.0 gamma: 0.5 acc_sum: 1.9530638852672753 acc_diff: 0.

pos: 1024.0 neg: 0.125 gamma: 0.5 acc_sum: 2.0 acc_diff: 0.0
pos: 1024.0 neg: 0.25 gamma: 0.5 acc_sum: 2.0 acc_diff: 0.0
pos: 1024.0 neg: 0.5 gamma: 0.5 acc_sum: 2.0 acc_diff: 0.0
pos: 1024.0 neg: 1.0 gamma: 0.5 acc_sum: 2.0 acc_diff: 0.0
pos: 1024.0 neg: 2.0 gamma: 0.5 acc_sum: 2.0 acc_diff: 0.0
pos: 1024.0 neg: 4.0 gamma: 0.5 acc_sum: 2.0 acc_diff: 0.0
pos: 1024.0 neg: 8.0 gamma: 0.5 acc_sum: 2.0 acc_diff: 0.0
pos: 1024.0 neg: 16.0 gamma: 0.5 acc_sum: 2.0 acc_diff: 0.0
pos: 1024.0 neg: 32.0 gamma: 0.5 acc_sum: 2.0 acc_diff: 0.0
pos: 1024.0 neg: 64.0 gamma: 0.5 acc_sum: 2.0 acc_diff: 0.0
pos: 1024.0 neg: 128.0 gamma: 0.5 acc_sum: 2.0 acc_diff: 0.0
pos: 1024.0 neg: 256.0 gamma: 0.5 acc_sum: 2.0 acc_diff: 0.0
pos: 1024.0 neg: 512.0 gamma: 0.5 acc_sum: 2.0 acc_diff: 0.0
pos: 1024.0 neg: 1024.0 gamma: 0.5 acc_sum: 2.0 acc_diff: 0.0
pos: 1024.0 neg: 2048.0 gamma: 0.5 acc_sum: 2.0 acc_diff: 0.0
pos: 1024.0 neg: 4096.0 gamma: 0.5 acc_sum: 2.0 acc_diff: 0.0
pos: 1024.0 neg: 8192.0 gamma: 0.5 

In [26]:
#load in after save
info = pd.DataFrame.from_csv("info_SVMplus.csv")
#info.iloc[:,3] = np.round(info.iloc[:,3],decimals = 2)
#info.iloc[:,4] = np.round(info.iloc[:,4],decimals = 2)
reduced_info = info.drop_duplicates(["3","4"])
selected = reduced_info.loc[(reduced_info.iloc[:,3]>max(info.iloc[:,3])-0.6)&(reduced_info.iloc[:,4]<0.06),:]
print(selected.shape)
print(selected)

(7, 5)
            0         1    2         3         4
0    0.000031  0.000031  0.1  1.765188  0.039160
1    0.000031  0.000061  0.1  1.814324  0.033502
9    0.000031  0.015625  0.1  1.811495  0.058070
96   0.000244  0.000244  0.1  1.751489  0.052859
130  0.000488  0.001953  0.1  1.836063  0.055241
963  0.000031  0.000122  0.2  1.800625  0.047201
970  0.000031  0.015625  0.2  1.825194  0.044372


C:\Users\KOU TIANQI\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


## Training Accuracy and Confusion Matrix

In [27]:
result_combo = []

for j in np.arange(0,len(selected)):
    
    C_pos,C_neg,gamma = selected.iloc[j,0:3]
    w,b,w1,w2,d1,d2 = SVMplus(train,C_pos,C_neg,gamma)
    count = 0
    result = []
    
    for i in range(0,train.shape[0]):
        if train.iloc[i][0] == "GP" :
            result.append(sum(train.iloc[i][1:-1]*w)+b+sum(train.iloc[i][1:-1]*w1)+d1)
        else:
            result.append(sum(train.iloc[i][1:-1]*w)+b+sum(train.iloc[i][1:-1]*w2)+d2)
    
    result = np.array(result)/abs(np.array(result))
    result_combo.append(result)
    print(j+1,":training confusion matrix:\n",confusion_matrix(train["target"],result))
    print("%f" % ((confusion_matrix(train["target"],result)[0,0]+confusion_matrix(train["target"],result)[1,1])/train.shape[0]),"%")

voted = np.squeeze(np.asarray(sum(np.matrix(result_combo)/abs(np.matrix(result_combo)))))/abs(np.squeeze(np.asarray(sum(np.matrix(result_combo)/abs(np.matrix(result_combo))))))
print("voted confusion matrix:\n",confusion_matrix(train["target"],voted))
print("%f" % ((confusion_matrix(train["target"],voted)[0,0]+confusion_matrix(train["target"],voted)[1,1])/train.shape[0]),"%")
print("-1:","%f" % (confusion_matrix(train["target"],voted)[0,0]/sum(confusion_matrix(train["target"],voted)[0,:])),"%")
print("1:","%f" % (confusion_matrix(train["target"],voted)[1,1]/sum(confusion_matrix(train["target"],voted)[1,:])),"%")

1 :training confusion matrix:
 [[83  9]
 [10 63]]
0.884848 %
2 :training confusion matrix:
 [[85  7]
 [ 8 65]]
0.909091 %
3 :training confusion matrix:
 [[86  6]
 [ 9 64]]
0.909091 %
4 :training confusion matrix:
 [[83  9]
 [11 62]]
0.878788 %
5 :training confusion matrix:
 [[87  5]
 [ 8 65]]
0.921212 %
6 :training confusion matrix:
 [[85  7]
 [ 9 64]]
0.903030 %
7 :training confusion matrix:
 [[86  6]
 [ 8 65]]
0.915152 %
voted confusion matrix:
 [[85  7]
 [ 8 65]]
0.909091 %
-1: 0.923913 %
1: 0.890411 %


## Test Accuracy and Confusion Matrix

#### Check results

In [28]:
result_combo_test = []

for j in np.arange(0,len(selected)):
    
    C_pos,C_neg,gamma = selected.iloc[j,0:3]
    result = []
    
    for i in range(0,train.shape[0]):
        X_test = train.iloc[i,0:-1]
        w,b,w1,w2,d1,d2 = SVMplus(train.drop(train.index[i]),C_pos,C_neg,gamma)
        if X_test[0] == "GP" :
            result.append(sum(X_test[1:]*w)+b+sum(X_test[1:]*w1)+d1)
        elif X_test[0] == "MS":
            result.append(sum(X_test[1:]*w)+b+sum(X_test[1:]*w2)+d2)
    
    result = np.array(result)/abs(np.array(result))
    result_combo_test.append(result)
    print(j+1,":test confusion matrix:\n",confusion_matrix(train["target"],result))
    print("%f" % ((confusion_matrix(train["target"],result)[0,0]+confusion_matrix(train["target"],result)[1,1])/train.shape[0]),"%")

voted_test = np.squeeze(np.asarray(sum(np.matrix(result_combo_test)/abs(np.matrix(result_combo_test)))))/abs(np.squeeze(np.asarray(sum(np.matrix(result_combo_test)/abs(np.matrix(result_combo_test))))))
print("voted confusion matrix:\n",confusion_matrix(train["target"],voted_test))
print("%f" % ((confusion_matrix(train["target"],voted_test)[0,0]+confusion_matrix(train["target"],voted_test)[1,1])/train.shape[0]),"%")
print("-1:","%f" % (confusion_matrix(train["target"],voted_test)[0,0]/sum(confusion_matrix(train["target"],voted_test)[0,:])),"%")
print("1:","%f" % (confusion_matrix(train["target"],voted_test)[1,1]/sum(confusion_matrix(train["target"],voted_test)[1,:])),"%")

1 :test confusion matrix:
 [[79 13]
 [37 36]]
0.696970 %
2 :test confusion matrix:
 [[82 10]
 [33 40]]
0.739394 %
3 :test confusion matrix:
 [[75 17]
 [28 45]]
0.727273 %
4 :test confusion matrix:
 [[78 14]
 [37 36]]
0.690909 %
5 :test confusion matrix:
 [[81 11]
 [32 41]]
0.739394 %
6 :test confusion matrix:
 [[82 10]
 [33 40]]
0.739394 %
7 :test confusion matrix:
 [[77 15]
 [29 44]]
0.733333 %
voted confusion matrix:
 [[79 13]
 [32 41]]
0.727273 %
-1: 0.858696 %
1: 0.561644 %
